# Learn about the optimization loop 了解优化循环

# Optimizing Model Parameters 优化模型参数

现在我们有了模型和数据，是时候通过优化数据上的参数来训练、验证和测试我们的模型了。训练模型是一个迭代过程；在每次迭代中，模型都会对输出进行猜测，计算其猜测中的误差（*损失*），收集相对于其参数的导数的误差（如我们在[上一节](https://pytorch.org/tutorials/beginner/basics/autograd_tutorial.html)中看到的），并使用梯度下降**优化**这些参数。有关此过程的更详细演练，请观看[3Blue1Brown 的反向传播](https://www.youtube.com/watch?v=tIeHLnjs5U8)有关视频。

## Prerequisite Code 前置代码

我们加载前面有关[数据集和数据加载器](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) 以及[构建模型](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)的代码。

In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Setting hyperparameters 设置超参数

超参数是可调整的参数，可让您控制模型优化过程。不同的超参数值会影响模型训练和收敛速度（[阅读](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)有关超参数调整的更多信息）

我们定义以下训练超参数：

-  **Number of Epochs** - 整个训练数据集通过网络的次数。
-  **Batch Size** - 每个Epoch模型看到的数据样本数量。迭代完成一个epoch所需的批次数。
-  **Learning Rate**- 模型在搜索可产生更高模型精度的最佳权重时匹配的步长大小。值越小意味着模型需要更长的时间才能找到最佳权重。较大的值可能会导致模型跳过并错过最佳权重，从而在训练期间产生不可预测的行为。

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Add an optimization loop 添加优化循环

一旦我们设置了超参数，我们就可以使用优化循环来训练和优化我们的模型。优化循环的每次迭代称为一个**epoch**。

每个 epoch由两个主要部分组成：

-  **The Train Loop** - 迭代训练数据集并尝试收敛到最佳参数。
-  **The Validation/Test Loop** - 迭代测试数据集以检查模型性能是否有所改善。

让我们简单熟悉一下训练循环中使用的一些概念。向前跳转查看优化循环的[完整实现](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#full-impl-label)。

### Add a loss function 添加损失函数

当提供一些训练数据时，我们未经训练的网络可能不会给出正确的答案。**损失函数**衡量的是得到的结果与目标值的不相似程度，它是我们在训练时想要最小化的损失函数。为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实数据标签值进行比较。

常见的损失函数包括：
- [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)（Mean Square Error 均方误差） 用于回归任务
- [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)（Negative Log Likelihood 负对数似然） 用于分类
- [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) 结合了`nn.LogSoftmax`和`nn.NLLLoss`。

我们将模型的输出 logits 传递给`nn.CrossEntropyLoss`，这将标准化 logits 并计算预测误差。

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

###  Optimization pass 优化传递

优化是调整模型参数以减少每个训练步骤中模型误差的过程。**Optimization algorithms**定义了如何执行此过程（在本例中我们使用 *Stochastic Gradient Descent* *随机梯度下降*）。所有优化逻辑都封装在`optimizer`对象中。这里，我们使用SGD优化器；此外，PyTorch 中还有许多[不同的优化器](https://pytorch.org/docs/stable/optim.html) ，例如 ADAM 和 RMSProp，它们可以更好地处理不同类型的模型和数据。

注册需要训练的模型参数，并传入学习率超参数。我们通过这种方式，来初始化优化器。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

在训练循环中，优化分三个步骤进行：

- 调用`optimizer.zero_grad()`重置模型参数的梯度。默认情况下渐变相加；为了防止重复计算，我们在每次迭代时明确地将它们归零。
- 通过调用`loss.backward()`来反向传播预测损失。PyTorch 存储每个参数的损失梯度。
- 一旦我们有了梯度，通过后向传递中收集的梯度，我们就可以调用`optimizer.step()`来调整参数。

## Full Implementation 全面实施

我们定义了`train_loop`优化代码的循环，`test_loop`根据我们的测试数据评估模型的性能。

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} {current:>5d}/{size:>5d}")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100 * correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化损失函数和优化器，并将其传递给`train_loop`和`test_loop`。请随意增加epoch数来跟踪模型改进的性能。

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1} \n ----------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(train_dataloader, model, loss_fn)
print("Done!")

Epoch 1 
 ----------
loss: 2.301911     0/60000
loss: 2.292816  6400/60000
loss: 2.287881 12800/60000
loss: 2.287051 19200/60000
loss: 2.255377 25600/60000
loss: 2.253076 32000/60000
loss: 2.260443 38400/60000
loss: 2.247314 44800/60000
loss: 2.241305 51200/60000
loss: 2.210870 57600/60000
Test Error: 
 Accuracy: 36.2%, Avg loss: 2.231190 

Epoch 2 
 ----------
loss: 2.239081     0/60000
loss: 2.230416  6400/60000
loss: 2.228071 12800/60000
loss: 2.236214 19200/60000
loss: 2.153478 25600/60000
loss: 2.158298 32000/60000
loss: 2.178127 38400/60000
loss: 2.160697 44800/60000
loss: 2.154827 51200/60000
loss: 2.075521 57600/60000
Test Error: 
 Accuracy: 36.6%, Avg loss: 2.133443 

Epoch 3 
 ----------
loss: 2.146330     0/60000
loss: 2.128829  6400/60000
loss: 2.128925 12800/60000
loss: 2.157317 19200/60000
loss: 1.988493 25600/60000
loss: 2.024168 32000/60000
loss: 2.047211 38400/60000
loss: 2.034990 44800/60000
loss: 2.041876 51200/60000
loss: 1.895464 57600/60000
Test Error: 
 Accuracy:

您可能已经注意到该模型最初不是很好（没关系！）。尝试运行循环更多的 `epochs` 或将 `learning_rate` 调整为更大的数字。也可能是这样的情况，我们选择的模型配置可能不是解决此类问题的最佳配置（事实并非如此）。后续课程将更深入地研究适用于视觉问题的模型形状。

保存模型 
------------- 
当您对模型的性能感到满意时，可以使用 `torch.save` 保存它。 PyTorch 模型将学习到的参数存储在internal state dictionar内部状态字典中，称为 `state_dict` 。这些可以通过 `torch.save` 方法保存：

In [7]:
torch.save(model.state_dict(), "data/model.pth")

print("Save PyToch Model State to model.pth")

Save PyToch Model State to model.pth
